In [ ]:
import pandas as pd
import numpy as np
import glob
from pandas import DataFrame
import random
import functools
import math
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from math import radians
from math import pi
import time
import os
import matplotlib as mpl

mpl.rcParams.update(mpl.rcParamsDefault)

In [ ]:
%run ./CommonFunctions_OthersGeneral.ipynb  ## This is important for library import, getting the path and runlist

# Normal Training

In [ ]:
def group_mean():
    runMean = []
    runStd =[]
    
    for num in runlist:
        all_images = []
        filename_screenshot = (f"Table_Run{num}.csv")
        run_table = pd.read_csv(dir6 + filename_screenshot, usecols = ['ScreenShotPath'])
        
        for row in run_table.itertuples():
            all_images.append(mpimg.imread(row.ScreenShotPath))
        all_images = np.array(all_images)
    
        # Full RGB mean
        full_rgb_mean = np.mean(all_images, axis=(0,1))
        full_rgb_std = np.std(all_images, axis=(0,1))    
        
        runMean.append(full_rgb_mean)
        runStd.append(full_rgb_std)
        
    return runMean,runStd

In [ ]:
## To generate the list before feeding into datagenerator ##

def datagenerator_prep1_extractdata(runMean,runStd,runlist,grid_division,mode):
    
    if mode == 'training' or mode == 'learning':
        chosen_angle = ['45°','135°','225°','315°'] # X
        #chosen_angle = ['315°','0°/360°','45°','90°'] # Horizontal Upper
        #chosen_angle = ['315°','0°/360°','45°'] # Horizontal Upper, just 3 angles
        #chosen_angle = ['0°/360°','45°','90°','135°'] # Vertical right
    elif mode == 'testing':
        chosen_angle = ['0°/360°','90°','180°','270°'] # +
        #chosen_angle = ['135°','180°','225°','270°'] # Horizontal lower
        #chosen_angle = ['135°','180°','225°'] # Horizontal lower, just 3 angles
        #chosen_angle = ['180°','225°','270°','315°'] # Vertical left
    else: ## In case user forget to define the mode, the program will be halted
        raise ValueError('Please select mode of paradigm')
        pass
    
    run_data = []
    
    index = 0
    
    for num in runlist: ## Do for every run ##
        XY = [] ## The existing XY positions ##
        XYnew = [] ## The newly divided XY positions ##
        chosenCoordinates = [] ## To choose an array of old position that matches most closely with the new positions ##
        
        filename_screenshot = (f"Table_Run{num}.csv")
        run_table = pd.read_csv(dir6 + filename_screenshot, usecols = ['Angle','X','Y','ScreenShotPath']) ## To read at the right csv table ##
        maxX = max(run_table['X']) ## finding the maximum X in old positions ##
        minX = min(run_table['X'])
        maxY = max(run_table['Y']) ## finding the maximum Y in old positions ##
        minY = min(run_table['Y'])

        for rt in run_table.itertuples(): ## To make X,Y pairing tuple list ##
            label_x = rt.X
            label_y = rt.Y
            XY.append((label_x, label_y)) ## To make each row of the X and Y into a tuple pair
        XY = list(set(XY)) ## To remove duplication in list by converting it to set to speed up comparison later by shrinking the size 8 times ##

        X = np.linspace(0,maxX,grid_division) ## To get the new X portion of division ##
        Y = np.linspace(0,maxY,grid_division) ## To get the new X portion of division ##
        mesh = np.meshgrid(X,Y) # To make even distribution across the map
        X = mesh[0].flatten() # X-axis
        Y = mesh[1].flatten() # Y-axis
        XYnew = [[x, y] for x, y in zip(X, Y)] ## To convert each X and Y (in ascending order) into tuple pairs in one list: [(X1,Y1),(X2,Y2)...]
            
        for grid in XYnew: ## For each tuple XY pair in XYnew list
            def distance(point_a, point_b): ## To define Xold - Xnew and Yold-Ynew (Teaching the program to do maths the way you define) ##
                x0, y0 = point_a
                x1, y1 = point_b
                return math.fabs(x0 - x1) + math.fabs(y0 - y1)
        
            def nearest(grid, XY): ## To get the smallest value from Xold - Xnew and Yold-Ynew ##
                distance_from_point = functools.partial(distance, grid)
                return min(XY, key=distance_from_point)
            chosenCoordinates.append(nearest(grid,XY)) ## To store the coordinates closest to each new position in an array of tuple pairings
        
        data = []
    
        for row in run_table.itertuples():
            for x,y in chosenCoordinates:
                if row.X == x and row.Y == y and any(row.Angle in angle for angle in chosen_angle):
                    newArray = row.Index
                    data.append(newArray) ## To form a list of indices of training data
        data = run_table.iloc[data] # MF: Index into dataframe with chosen indices
        
        ## To rescale the X and Y
        downScalingX_column = []
        downScalingY_column = []
        runMean_column = []
        runStd_column = []
        
        for i in data.itertuples():
            downScalingX = (i.X - minX)/(maxX - minX)
            downScalingX_column.append(downScalingX)
            downScalingY = (i.Y - minY)/(maxY - minY)
            downScalingY_column.append(downScalingY)
            runMeanValue = runMean[index]
            runMean_column.append(runMeanValue)
            runStdValue = runStd[index]
            runStd_column.append(runStdValue)
            
        
        data = data.assign(downScalingX = downScalingX_column)
        data = data.assign(downScalingY = downScalingY_column)
        data = data.assign(runMeanValue = runMean_column)
        data = data.assign(runStdValue = runStd_column)
        
        run_data.append(data)
        
        index += 1

    return run_data

In [ ]:
# To perform random check on the data (Original Image, Normalized Images, Histogram of these images)
# Figure 2.6

def normalized_illustration_randomcheck(run_data,samplingsize,mode):
    data = random.choice(run_data)
    row = data.sample(samplingsize)

    oriImage_all = []
    firstNorm_all = []
    secondNorm_all = []
    thirdNorm_all = []
    
    nt = time.strftime("%d-%m-%Y_%H%M")
        
    folder = dir22 + '/' + test_runNum + '-' + mode + '_randomNormCheck_' + nt
    if not os.path.exists(folder):
        os.makedirs(folder)

    for i in row.itertuples():
        img = mpimg.imread(i.ScreenShotPath)
        oriImage_all.append(img)

        image = (img - i.runMeanValue) / (i.runStdValue) #+ 0.0001) # Group Normalisation # May get the error 'can only concatenate list (not "float") to list' if i add 0.0001 although the original intention of doing this is to avoid zero error during model training
        image = image.astype(np.float32)
        firstNorm_all.append(image)
        
        
        # ImageNet Technique (Not necessary, because next step makes it into 0 to 1. 0 to 1 is preferred because negative values will become 0 weight in relu of Conv2D)
        # But just placed in since previous testings take this algorithm into account, and no more time to rerun everything before report submission
        # Advised to drop it if anyone from the lab is using my code unless your model activation does not change negative values into 0, then maybe you can try using this and cut out next standardization step
        image /= 127.5 # To downscale it from (0-255) to 
        image -= 1.    # (-1 to 1)
        secondNorm_all.append(image)

        image = (image - np.min(image))/(np.max(image) - np.min(image)) # To convert the range into (0 to 1)
        thirdNorm_all.append(image)

    oriImage_all= np.asarray(oriImage_all)
    thirdNorm_all= np.asarray(thirdNorm_all)
    
    imageLen = list(range(0,(oriImage_all.shape[0]),1))
    
    for i in imageLen:
        fig, axes = plt.subplots(2,2, figsize=(15,8))#,constrained_layout=True)
        axes = axes.flatten()

        for idx, ax in enumerate(axes):
            if idx == 0:
                dataname = 'OriginalImage'
                img = oriImage_all[i]
                ax.set_title(dataname, pad=20, fontsize=14, fontweight='bold')                          
                ax.matshow(img)
            elif idx == 1:
                dataname = 'OriginalImage_Histogram'
                img = oriImage_all[i]
                ax.set_title(dataname, pad=20, fontsize=14, fontweight='bold')
                ax.xaxis.set_ticks_position('top')
                ax.hist(img.flatten())
            if idx == 2:
                dataname = 'NormalizedImage' #'3rd_Normalize'
                img = thirdNorm_all[i] #[idx]
                ax.set_title(dataname, pad=20, fontsize=14, fontweight='bold')                
                ax.matshow(img)
            elif idx == 3:
                dataname = 'NormalizedImage_Histogram' #'3rd_Normalize_Histogram'
                img = thirdNorm_all[i]
                ax.set_title(dataname, pad=20, fontsize=14, fontweight='bold')
                ax.xaxis.set_ticks_position('top')
                #ax.set_xticks([0.0,0.0,0.2,0.4,0.6,0.8,1.0,1.0])
                #ax.set_xticklabels(['0.0','0.0','0.2','0.4','0.6','0.8','1.0','1.0'])
                ax.hist(img.flatten())
                
        #GeneralTitle = 'Random Visualization for Normalization' #(f"Run{num} n\n\n\n\n\n\n\n\n\n\n\n")
        #fig.suptitle((GeneralTitle + "\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n"), 
        #             fontsize=14, fontweight='bold', ha='center', va='top')
        fig.tight_layout()
        fig.savefig(folder + '/' + test_runNum + '-' + mode + '_randomNormCheck' + str(i) + '.png')
        fig.show()

In [ ]:
## Extension for report figure
def normalized_illustration_randomcheck_Figure4_1(run_data,samplingsize,mode):
    data = random.choice(run_data)
    row = data.sample(samplingsize)

    oriImage_all = []
    mean_all = []
    meanStd_all = []
    thirdNorm_all = []
    
    nt = time.strftime("%d-%m-%Y_%H%M")
        
    folder = dir22 + '/' + test_runNum + '-' + mode + '_randomNormCheck_' + nt
    if not os.path.exists(folder):
        os.makedirs(folder)

    for i in row.itertuples():
        mean = i.runMeanValue
        std = i.runStdValue
        
        img = mpimg.imread(i.ScreenShotPath)
        oriImage_all.append(img)
        
        imageM = (img - i.runMeanValue)
        imageM = imageM.astype(np.float32)
        mean_all.append(imageM)

        imageMS = (img - i.runMeanValue) / (i.runStdValue)
        imageMS = imageMS.astype(np.float32)
        meanStd_all.append(imageMS)
        
        image = (img - i.runMeanValue) / (i.runStdValue)
        image = image.astype(np.float32)
        image /= 127.5 # To downscale it from (0-255) to 
        image -= 1.    # (-1 to 1)
        image = (image - np.min(image))/(np.max(image) - np.min(image)) # To convert the range into (0 to 1)
        thirdNorm_all.append(image)
        

    oriImage_all= np.asarray(oriImage_all)
    mean_all= np.asarray(mean_all)
    meanStd_all= np.asarray(meanStd_all)
    thirdNorm_all= np.asarray(thirdNorm_all)
    
    imageLen = list(range(0,(oriImage_all.shape[0]),1))
    
    for i in imageLen:
        fig, axes = plt.subplots(2,2, figsize=(15,8))
        axes = axes.flatten()

        for idx, ax in enumerate(axes):
            if idx == 0:
                dataname = 'Original Image'
                img = oriImage_all[i]
                ax.set_title(dataname, pad=20, fontsize=14, fontweight='bold')                          
                ax.matshow(img)
            elif idx == 1:
                dataname = 'Original Image - Mean'
                img = mean_all[i]
                ax.set_title(dataname, pad=20, fontsize=14, fontweight='bold')                
                ax.matshow(img)
            elif idx == 2:#4:
                dataname = '(Original Image - Mean)/Std'
                img = meanStd_all[i]
                ax.set_title(dataname, pad=20, fontsize=14, fontweight='bold')                
                ax.matshow(img)
            elif idx == 3:
                dataname = 'Normalized Image'
                img = thirdNorm_all[i]
                ax.set_title(dataname, pad=20, fontsize=14, fontweight='bold')                
                ax.matshow(img)
            
        fig.tight_layout()
        fig.savefig(folder + '/' + test_runNum + '-' + mode + '_randomNormCheck' + str(i) + '.png')
        fig.show()

In [ ]:
## Multiple images each batch depend on specified batch size
def datagenerator(run_data, samplingsize, angles):
    while True:
        if angles == 'yes':
            data = random.choice(run_data)
            row = data.sample(samplingsize)

            images = []
            xy = []
            angles = []

            for i in row.itertuples():
                img = mpimg.imread(i.ScreenShotPath)
                image = (img - i.runMeanValue) / (i.runStdValue) #+ 0.0001) # Group Normalisation # May get the error 'can only concatenate list (not "float") to list' if i add 0.0001 although the original intention of doing this is to avoid zero error during model training
                x = i.downScalingX
                y = i.downScalingY
                image = image.astype(np.float32)
                # ImageNet Technique (Not necessary, because next step makes it into 0 to 1. 0 to 1 is preferred because negative values will become 0 weight in relu of Conv2D)
                # But just placed in since previous testings take this algorithm into account, and no more time to rerun everything before report submission
                # Advised to drop it if anyone from the lab is using my code unless your model activation does not change negative values into 0, then maybe you can try using this and cut out next standardization step
                image /= 127.5 # To downscale it from (0-255) to 
                image -= 1.    # (-1 to 1)
                image = (image - np.min(image))/(np.max(image) - np.min(image)) # To convert the range into (0 to 1)
                images.append(image)
                labels = np.array((x,y))
                xy.append(labels)

                angle = i.Angle 
                if len(angle) >= 3 and len(angle) <= 4:
                    angle = angle[:-1]
                elif len(angle) == 7:
                    angle = angle[0]   
                angle = radians(int(angle)) # To convert degree into radians ##angle = angle / pi # To further convert into pi radians           
                angles.append(angle)

            images= np.asarray(images)
            xy =  np.asarray(xy)
            angles =  np.asarray(angles)

            yield images, [xy, angles]
        
        elif angles == 'no':
            data = random.choice(run_data)
            row = data.sample(samplingsize)

            images = []
            xy = []

            for i in row.itertuples():
                img = mpimg.imread(i.ScreenShotPath)
                image = (img - i.runMeanValue) / (i.runStdValue) #+ 0.0001) # Group Normalisation # May get the error 'can only concatenate list (not "float") to list' if i add 0.0001 although the original intention of doing this is to avoid zero error during model training
                x = i.downScalingX
                y = i.downScalingY
                image = image.astype(np.float32)
                # ImageNet Technique (Not necessary, because next step makes it into 0 to 1. 0 to 1 is preferred because negative values will become 0 weight in relu of Conv2D)
                # But just placed in since previous testings take this algorithm into account, and no more time to rerun everything before report submission
                # Advised to drop it if anyone from the lab is using my code unless your model activation does not change negative values
                image /= 127.5 # To downscale it from (0-255) to 
                image -= 1.    # (-1 to 1)
                image = (image - np.min(image))/(np.max(image) - np.min(image)) # To convert the range into (0 to 1)
                images.append(image)
                labels = np.array((x,y))
                xy.append(labels)

            images= np.asarray(images)
            xy =  np.asarray(xy)

            yield images, xy


# Model Verification

In [ ]:
# To preprocess all and distribute for train and test as percentage
# Section 3.2.3 of report

## To generate the list before feeding into datagenerator ##


def dataProcessAll_PercentageTest_Step1(runlist,grid_division,train_percentage):
    
    run_data = []
    
    index = 0
    
    for num in runlist: ## Do for every run ##
        all_images = []
        XY = [] ## The existing XY positions ##
        XYnew = [] ## The newly divided XY positions ##
        chosenCoordinates = [] ## To choose an array of old position that matches most closely with the new positions ##
        
        filename_screenshot = (f"Table_Run{num}.csv")
        run_table = pd.read_csv(dir6 + filename_screenshot, usecols = ['Angle','X','Y','ScreenShotPath']) ## To read at the right csv table ##
        
        maxX = max(run_table['X']) ## finding the maximum X in old positions ##
        minX = min(run_table['X'])
        maxY = max(run_table['Y']) ## finding the maximum Y in old positions ##
        minY = min(run_table['Y'])

        for rt in run_table.itertuples(): ## To make X,Y pairing tuple list ##
            all_images.append(mpimg.imread(rt.ScreenShotPath))
            label_x = rt.X
            label_y = rt.Y
            XY.append((label_x, label_y)) ## To make each row of the X and Y into a tuple pair
        XY = list(set(XY)) ## To remove duplication in list by converting it to set to speed up comparison later by shrinking the size 8 times ##
        all_images = np.array(all_images)
        
        # Full RGB mean
        full_rgb_mean = np.mean(all_images, axis=(0,1)) 
        full_rgb_std = np.std(all_images, axis=(0,1))
        
        X = np.linspace(0,maxX,grid_division) ## To get the new X portion of division ##
        Y = np.linspace(0,maxY,grid_division) ## To get the new X portion of division ##
        mesh = np.meshgrid(X,Y) # To make even distribution across the map
        X = mesh[0].flatten() # X-axis
        Y = mesh[1].flatten() # Y-axis
        XYnew = [[x, y] for x, y in zip(X, Y)] ## To convert each X and Y (in ascending order) into tuple pairs in one list: [(X1,Y1),(X2,Y2)...]
            
        for grid in XYnew: ## For each tuple XY pair in XYnew list
            def distance(point_a, point_b): ## To define Xold - Xnew and Yold-Ynew (Teaching the program to do maths the way you define) ##
                x0, y0 = point_a
                x1, y1 = point_b
                return math.fabs(x0 - x1) + math.fabs(y0 - y1)
        
            def nearest(grid, XY): ## To get the smallest value from Xold - Xnew and Yold-Ynew ##
                distance_from_point = functools.partial(distance, grid)
                return min(XY, key=distance_from_point)
            chosenCoordinates.append(nearest(grid,XY)) ## To store the coordinates closest to each new position in an array of tuple pairings
        
        data = []
    
        for row in run_table.itertuples():
            for x,y in chosenCoordinates:
                if row.X == x and row.Y == y:
                    newArray = row.Index
                    data.append(newArray) ## To form a list of indices of training data
        data = run_table.iloc[data] # MF: Index into dataframe with chosen indices
        
        ## To rescale the X and Y
        downScalingX_column = []
        downScalingY_column = []
        runMean_column = []
        runStd_column = []
        #image_column = []
        #downScalingAngle_column = []
        #downScalingXYlabel_column = []

        for i in data.itertuples():
            downScalingX = (i.X - minX)/(maxX - minX)
            downScalingX_column.append(downScalingX)
            downScalingY = (i.Y - minY)/(maxY - minY)
            downScalingY_column.append(downScalingY)

            runMeanValue = full_rgb_mean
            runMean_column.append(runMeanValue)
            runStdValue = full_rgb_std
            runStd_column.append(runStdValue)

            #label_xy = np.array((downScalingX,downScalingY))
            #downScalingXYlabel_column.append(label_xy)

            #img = mpimg.imread(i.ScreenShotPath)
            #image = (img - runMeanValue) / (runStdValue) #+ 0.0001) # Group Normalisation # May get the error 'can only concatenate list (not "float") to list' if i add 0.0001 although the original intention of doing this is to avoid zero error during model training
            #image = image.astype(np.float32)
            #image /= 127.5 # To downscale it from (0-255) to 
            #image -= 1.    # (-1 to 1)
            #image = (image - np.min(image))/(np.max(image) - np.min(image)) # To convert the range into (0 to 1)
            #image = np.asarray(image)
            #image_column.append(image)

            #angle = i.Angle 
            #if len(angle) >= 3 and len(angle) <= 4:
            #    angle = angle[:-1]
            #elif len(angle) == 7:
            #    angle = angle[0]           
            #downScalingAngle = radians(int(angle)) # To convert degree into radians ##angle = angle / pi # To further convert into pi radians   
            #downScalingAngle_column.append(downScalingY)

        data = data.assign(downScalingX = downScalingX_column)
        data = data.assign(downScalingY = downScalingY_column)
        data = data.assign(runMeanValue = runMean_column)
        data = data.assign(runStdValue = runStd_column)
        #data = data.assign(downScalingXYlabel = downScalingXYlabel_column)
        #data = data.assign(normalisedImageNP = image_column)
        #data = data.assign(downScalingAngle = downScalingAngle_column)
        data = data.sample(frac=1).reset_index(drop=True) # To shuffle sequence between a run
        
        run_data.append(data)
        
        index += 1

    all_rd = []
    rd_len = list(range(0,len(run_data),1))

    for i in rd_len:
        item = eval(f"run_data[{i}]")
        all_rd.append(item)
    #print(type(all_rd),all_rd)

    new_run_data = pd.concat(all_rd)
    #new_run_data = new_run_data.sample(frac=1).reset_index(drop=True) # To shuffle sequence across runs
    
    selected_datafeed_train = round(int(len(new_run_data) * train_percentage))
    training_data = new_run_data[:int(selected_datafeed_train)]
    testing_data = new_run_data[int(selected_datafeed_train):]
    

    return training_data,testing_data


########################################################################################


## Multiple images each batch depend on specified batch size
def datagenerator_PercentageTest_Step2(run_data, samplingsize, angles):
    while True:
        if angles == 'yes':
            row = run_data.sample(samplingsize)

            images = []
            xy = []
            angles = []

            for i in row.itertuples():
                img = mpimg.imread(i.ScreenShotPath)
                image = (img - i.runMeanValue) / (i.runStdValue) #+ 0.0001) # Group Normalisation # May get the error 'can only concatenate list (not "float") to list' if i add 0.0001 although the original intention of doing this is to avoid zero error during model training
                x = i.downScalingX
                y = i.downScalingY
                image = image.astype(np.float32)
                image /= 127.5 # To downscale it from (0-255) to 
                image -= 1.    # (-1 to 1)
                image = (image - np.min(image))/(np.max(image) - np.min(image)) # To convert the range into (0 to 1)
                images.append(image)
                labels = np.array((x,y))
                xy.append(labels)

                angle = i.Angle 
                if len(angle) >= 3 and len(angle) <= 4:
                    angle = angle[:-1]
                elif len(angle) == 7:
                    angle = angle[0]   
                angle = radians(int(angle)) # To convert degree into radians ##angle = angle / pi # To further convert into pi radians           
                angles.append(angle)

            images= np.asarray(images)
            xy =  np.asarray(xy)
            angles =  np.asarray(angles)

            yield images, [xy, angles]
        
        elif angles == 'no':
            row = run_data.sample(samplingsize)

            images = []
            xy = []

            for i in row.itertuples():
                img = mpimg.imread(i.ScreenShotPath)
                image = (img - i.runMeanValue) / (i.runStdValue) #+ 0.0001) # Group Normalisation # May get the error 'can only concatenate list (not "float") to list' if i add 0.0001 although the original intention of doing this is to avoid zero error during model training
                x = i.downScalingX
                y = i.downScalingY
                image = image.astype(np.float32)
                image /= 127.5 # To downscale it from (0-255) to 
                image -= 1.    # (-1 to 1)
                image = (image - np.min(image))/(np.max(image) - np.min(image)) # To convert the range into (0 to 1)
                images.append(image)
                labels = np.array((x,y))
                xy.append(labels)

            images= np.asarray(images)
            xy =  np.asarray(xy)

            yield images, xy

In [ ]:
# Section 3.2.2 of report (Shuffling of snapshots)

## To generate the list before feeding into datagenerator ##

def datagenerator_prep1_extractdata_Shuffle_250520(runMean,runStd,runlist,grid_division,mode):
    
    if mode == 'training' or mode == 'learning':
        chosen_angle = ['45°','135°','225°','315°'] # X
        #chosen_angle = ['315°','0°/360°','45°','90°'] # Horizontal Upper
        #chosen_angle = ['315°','0°/360°','45°'] # Horizontal Upper, just 3 angles
        #chosen_angle = ['0°/360°','45°','90°','135°'] # Vertical right
    elif mode == 'testing':
        chosen_angle = ['0°/360°','90°','180°','270°'] # +
        #chosen_angle = ['135°','180°','225°','270°'] # Horizontal lower
        #chosen_angle = ['135°','180°','225°'] # Horizontal lower, just 3 angles
        #chosen_angle = ['180°','225°','270°','315°'] # Vertical left
    else: ## In case user forget to define the mode, the program will be halted
        raise ValueError('Please select mode of paradigm')
        pass
    
    run_data = []
    
    index = 0
    
    for num in runlist: ## Do for every run ##
        XY = [] ## The existing XY positions ##
        XYnew = [] ## The newly divided XY positions ##
        chosenCoordinates = [] ## To choose an array of old position that matches most closely with the new positions ##
        
        filename_screenshot = (f"Table_Run{num}.csv")
        run_table = pd.read_csv(dir6 + filename_screenshot, usecols = ['Angle','X','Y','ScreenShotPath']) ## To read at the right csv table ##
        
        ##################################
        ## Shuffling here ##
        
        # run_table = run_table.sample(frac=1).reset_index(drop=True) # <<<< This only helps to reorder the whole row, and not row and every column to make mismatch
        # equal to run_table.reindex(np.random.permutation(run_table.index))
        # Difference between loc, iloc and ix: https://stackoverflow.com/questions/31593201/how-are-iloc-ix-and-loc-different
        
        angle = run_table['Angle']#.sample(frac=1).reset_index(drop=True)
        x = run_table['X']#.sample(frac=1).reset_index(drop=True)
        y = run_table['Y']#.sample(frac=1).reset_index(drop=True)
        ssPath = run_table['ScreenShotPath'].sample(frac=1).reset_index(drop=True)

        #shuffled = [angle, x, y, ssPath]
        #rt = pd.concat(shuffled)

        data = {'Angle':angle,'X':x,'Y':y,'ScreenShotPath':ssPath}
        df = pd.DataFrame(data, columns=['Angle','X','Y','ScreenShotPath'])
        
        run_table = df
        
        #import gc
        del df # deleting the variable df to save some RAM
        
        ##################################
        
        maxX = max(run_table['X']) ## finding the maximum X in old positions ##
        minX = min(run_table['X'])
        maxY = max(run_table['Y']) ## finding the maximum Y in old positions ##
        minY = min(run_table['Y'])

        for rt in run_table.itertuples(): ## To make X,Y pairing tuple list ##
            label_x = rt.X
            label_y = rt.Y
            XY.append((label_x, label_y)) ## To make each row of the X and Y into a tuple pair
        XY = list(set(XY)) ## To remove duplication in list by converting it to set to speed up comparison later by shrinking the size 8 times ##

        X = np.linspace(0,maxX,grid_division) ## To get the new X portion of division ##
        Y = np.linspace(0,maxY,grid_division) ## To get the new X portion of division ##
        mesh = np.meshgrid(X,Y) # To make even distribution across the map
        X = mesh[0].flatten() # X-axis
        Y = mesh[1].flatten() # Y-axis
        XYnew = [[x, y] for x, y in zip(X, Y)] ## To convert each X and Y (in ascending order) into tuple pairs in one list: [(X1,Y1),(X2,Y2)...]
            
        for grid in XYnew: ## For each tuple XY pair in XYnew list
            def distance(point_a, point_b): ## To define Xold - Xnew and Yold-Ynew (Teaching the program to do maths the way you define) ##
                x0, y0 = point_a
                x1, y1 = point_b
                return math.fabs(x0 - x1) + math.fabs(y0 - y1)
        
            def nearest(grid, XY): ## To get the smallest value from Xold - Xnew and Yold-Ynew ##
                distance_from_point = functools.partial(distance, grid)
                return min(XY, key=distance_from_point)
            chosenCoordinates.append(nearest(grid,XY)) ## To store the coordinates closest to each new position in an array of tuple pairings
        
        data = []
    
        for row in run_table.itertuples():
            for x,y in chosenCoordinates:
                if row.X == x and row.Y == y and any(row.Angle in angle for angle in chosen_angle):
                    newArray = row.Index
                    data.append(newArray) ## To form a list of indices of training data
        data = run_table.iloc[data] # MF: Index into dataframe with chosen indices
        
        ## To rescale the X and Y
        downScalingX_column = []
        downScalingY_column = []
        runMean_column = []
        runStd_column = []
        
        for i in data.itertuples():
            downScalingX = (i.X - minX)/(maxX - minX)
            downScalingX_column.append(downScalingX)
            downScalingY = (i.Y - minY)/(maxY - minY)
            downScalingY_column.append(downScalingY)
            runMeanValue = runMean[index]
            runMean_column.append(runMeanValue)
            runStdValue = runStd[index]
            runStd_column.append(runStdValue)
            
        
        data = data.assign(downScalingX = downScalingX_column)
        data = data.assign(downScalingY = downScalingY_column)
        data = data.assign(runMeanValue = runMean_column)
        data = data.assign(runStdValue = runStd_column)
        
        run_data.append(data)
        
        index += 1

    return run_data